[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Настройка параметров
В этом задании вам предстоит поэкспериментировать с параметрами вашей модели для сентимент-анализа. Все задания выполняются на том же датасете, что и на прошлой неделе.

## Инструкции

1. Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

2. Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

3. Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

4. Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

5. Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

Откатываем на нужную версию sklearn (иначе ответы не принимаются)

In [ ]:
!pip install scikit-learn==0.20.1

Импортируем библиотеки

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
nltk.download('movie_reviews')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

---
Загружаем данные

In [ ]:
negids = movie_reviews.fileids('neg') 
posids = movie_reviews.fileids('pos')
print('negative', len(negids), 'positive', len(posids))
neg_reviews = [' '.join(movie_reviews.words(fileids=[f])) for f in negids]
pos_reviews = [' '.join(movie_reviews.words(fileids=[f])) for f in posids]
corpus = neg_reviews + pos_reviews
labels = [0] * len(neg_reviews) + [1] * len(pos_reviews)
print('total', len(corpus))

negative 1000 positive 1000
total 2000


---
Оценим пайплайн из CountVectorizer и LogisticRegression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

vectorizer = CountVectorizer()
clf = LogisticRegression(max_iter=1000)
pline = Pipeline([("vectorizer", vectorizer), ("classifier", clf)])
res = cross_val_score(pline, corpus, labels, cv=5)
print(res.mean(), res.std())

0.8415000000000001 0.01677796173556255


---
Оценим пайплайн из TfidfVectorizer и LogisticRegression

In [ ]:
pline = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', clf)])
res = cross_val_score(pline, corpus, labels, cv=5)
print(res.mean(), res.std())

0.8210000000000001 0.004062019202317978


---
Проанализируем влияние параметра min_df (минимально необходимое количество вхождений слова в корпус документов) на точность классификации

In [ ]:
for min_df in (10, 50):
    pline = Pipeline([('vectorizer', CountVectorizer(min_df=min_df)), ('classifier', clf)])
    print(cross_val_score(pline, corpus, labels, cv=5).mean())

0.8390000000000001
0.813


---
Сравним различные алгоритмы классификации по точности

In [ ]:
classifiers = LogisticRegression(), LinearSVC(), SGDClassifier()
for clf in classifiers:
    # print(clf)
    pline = Pipeline([('vectorizer', CountVectorizer()), ('classifier', clf)])
    print(clf.__class__.__name__, cross_val_score(pline, corpus, labels, cv=5).mean())    

LogisticRegression 0.8415000000000001
LinearSVC 0.8325000000000001
SGDClassifier 0.7790000000000001


---
Векторизация с использованием списка стоп-слов из библиотеки nltk

In [ ]:
nltk_stopwords = stopwords.words('english')
vectorizer = CountVectorizer(stop_words=nltk_stopwords)
pline = Pipeline([('vectorizer', vectorizer), 
                  ('classifier', LogisticRegression(max_iter=1000))])
print(cross_val_score(pline, corpus, labels, cv=5).mean())    

0.8414999999999999


---
Векторизация с использованием списка стоп-слов из библиотеки sklearn

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
pline = Pipeline([('vectorizer', vectorizer), 
                  ('classifier', LogisticRegression(max_iter=1000))])
print(cross_val_score(pline, corpus, labels, cv=5).mean())    

0.8390000000000001


---
Векторизация с добавлением биграмм

In [ ]:
pline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))), 
                  ('classifier', LogisticRegression(max_iter=1000))])
print(cross_val_score(pline, corpus, labels, cv=5).mean())    

0.8525


---
Векторизация буквенных n-грамм

In [ ]:
pline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(3, 5), analyzer='char_wb')), 
                  ('classifier', LogisticRegression(max_iter=1000))])
print(cross_val_score(pline, corpus, labels, cv=5).mean())    

0.82
